In [ ]:
# Imports
import cv2
import os
import numpy as np
import pickle
from imutils import paths
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers.core import Flatten, Dense
from helpers import resize_to_fit
# Lista de dados e rótulos
dados=[]
rotulos=[]
# Pasta onde está as imagens de cada letra
pasta_base="base_letras"
# Retorna o texto do diretório
imagens=paths.list_images(pasta_base)
# Percorre cada arquivo em imagens
for arquivo in imagens:
    # Está pegando o nome de cada pasta como rótulo
    rotulo = arquivo.split(os.path.sep)[-2]
    # Abrindo as imagens
    imagem=cv2.imread(arquivo)
    # Convertendo para escala cinza
    imagem = cv2.cvtColor(imagem, cv2.COLOR_BGR2GRAY)
    # alterando o tamanho da imagem
    imagem = resize_to_fit(imagem, 20, 20)
    # Adicionando 1 dimensãoa imagem, característica do modelo
    imagem = np.expand_dims(imagem, axis=2)
    # Adiciando os dados as listas de imagens e rotulos
    rotulos.append(rotulo)
    dados.append(imagem)
# Normalizando as imagens e atrituindo um array a cada variável.
dados = np.array(dados, dtype="float")/ 255
rotulo = np.array(rotulos)

# Separação em dados de treino (75%) e teste (25%)
(X_train, X_test, Y_train, Y_test) = train_test_split(dados, rotulos, test_size=0.25, random_state=0)

# Converter com one-hot encoding
lb = LabelBinarizer().fit(Y_train)
Y_train=lb.transform(Y_train)
Y_test=lb.transform(Y_test)

# Salvar o LabelBinarizer em um arquivo com pickle
with open('rotulos_do_modelo.dat', 'wb') as arquivo_pickle:
    pickle.dump(lb, arquivo_pickle)

# Criando modelo
modelo = Sequential()
# Criando a 1 camada da rede neural
modelo.add(Conv2D(20, (5, 5), padding="same", input_shape=(20, 20, 1), activation="relu"))
modelo.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
# Criando a 2 camada da rede neural
modelo.add(Conv2D(50, (5, 5), padding="same", activation="relu"))
modelo.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
# Criando a 3 camada da rede neural
modelo.add(Flatten())
modelo.add(Dense(500, activation="relu"))
# Criando a camada de saida
# "26" é o número de saídas, pode ser alterado para retornar as possibilidades, neste caso é 26 lestras do alfabeto
modelo.add(Dense(26, activation="softmax"))
# Compilar todas as camadas
modelo.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

# Treinar a IA /epochs/ Quantas vezes passa pelo processo de treino, pode ser alterado de acordo com a necessidade /batch_size/ é o número de saídas citado acima
modelo.fit(X_train, Y_train, validation_data=(X_test, Y_test), batch_size=26, epochs=10, verbose=0)

# Salvar modelo em um arquivo
modelo.save("modelo_treinado.hdf5")